In [11]:
import utils
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset

In [21]:
class FakeNewsDataset(Dataset):
    def __init__(self):
        super(FakeNewsDataset, self).__init__()
        self.data = np.load("data/processed/data_bin12h_cut180_100w_20xu_50yu.npy", allow_pickle=True)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        if idx < 0:
            idx = len(self) + idx
        return torch.FloatTensor(self.data[idx][2]), torch.FloatTensor(self.data[idx][3]), self.data[idx][1]

fake_news_data = FakeNewsDataset()

In [22]:
x, y, label = fake_news_data[2]
print(x.shape, y.shape, label)

torch.Size([65, 122]) torch.Size([161, 50]) 0


In [27]:
torch.repeat_interleave(x.unsqueeze(0), 2, dim=0).shape

torch.Size([2, 65, 122])

In [37]:
class Capture(nn.Module):
    def __init__(self, feature_dim, feature_embedding_dim, lstm_hidden_dim, article_embedding_dim):
        super(Capture, self).__init__()
        self.input_embedding = nn.Sequential(
            nn.Linear(feature_dim, feature_embedding_dim),
            nn.Tanh(),
        )
        self.lstm = nn.LSTM(feature_embedding_dim, lstm_hidden_dim, batch_first=True)
        self.output_embedding = nn.Sequential(
            nn.Linear(lstm_hidden_dim, article_embedding_dim),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.input_embedding(x)
        _, (x, _) = self.lstm(x)
        x = self.output_embedding(x[-1])
        return x

class Score(nn.Module):
    def __init__(self, user_dim, user_embedding_dim):
        super(Score, self).__init__()
        self.user_embedding = nn.Sequential(
            nn.Linear(user_dim, user_embedding_dim), # add regularization
            nn.Tanh()
        )
        self.user_score = nn.Sequential(
            nn.Linear(user_embedding_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, y):
        y_hat = self.user_embedding(y)
        s = self.user_score(y_hat)
        p = s.mean(dim=1)
        return p, s, y_hat

class Integrate(nn.Module):
    def __init__(self, capture_dim):
        super(Integrate, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(capture_dim + 1, 1),
            #nn.Sigmoid() #  disabled, part of loss function
        )

    def forward(self, x, y):
        v = torch.cat((x, y), 1)
        v = self.net(v)
        return v


class CSI(nn.Module):
    def __init__(self, feature_dim, feature_embedding_dim, lstm_hidden_dim, article_embedding_dim,
                 user_dim, user_embedding_dim):
        super(CSI, self).__init__()
        self.capture = Capture(feature_dim, feature_embedding_dim, lstm_hidden_dim, article_embedding_dim)
        self.score = Score(user_dim, user_embedding_dim)
        self.integrate = Integrate(article_embedding_dim)

    def forward(self, x, y):
        x = self.capture(x)
        y, _, _ = self.score(y)
        l = self.integrate(x, y)
        return l


In [38]:
x = torch.repeat_interleave(x.unsqueeze(0), 2, dim=0)
y = torch.repeat_interleave(y.unsqueeze(0), 2, dim=0)

In [39]:
csi = CSI(122, 100, 100, 50, 50, 50)
csi(x, y)

tensor([[0.0041],
        [0.0041]], grad_fn=<AddmmBackward0>)